In [1]:
#importing desire libraries
import requests 
import json
import pandas as pd
from dotenv import load_dotenv
import os

In [2]:
#using the api
load_dotenv()
try:
    url = "https://timetable-lookup.p.rapidapi.com/airports/PTY/routes/"
    headers = {
    	"X-RapidAPI-Key": "traffic_key",
    	"X-RapidAPI-Host": "timetable-lookup.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        print(response.text)
    else:
        raise ValueError(f'Error: {response.status_code}')
except ValueError as ve:
    print(ve)

<Airports>
	<Routes>
		<NonStops>
			<NonStop From='ADZ' To='PTY' Carrier='CM' Code='' />
			<NonStop From='ADZ' To='PTY' Carrier='UA' Code='C' />
			<NonStop From='AMS' To='PTY' Carrier='AF' Code='C' />
			<NonStop From='AMS' To='PTY' Carrier='CM' Code='C' />
			<NonStop From='AMS' To='PTY' Carrier='KL' Code='' />
			<NonStop From='ASU' To='PTY' Carrier='CM' Code='' />
			<NonStop From='ASU' To='PTY' Carrier='TK' Code='C' />
			<NonStop From='ASU' To='PTY' Carrier='UA' Code='C' />
			<NonStop From='ATL' To='PTY' Carrier='AF' Code='C' />
			<NonStop From='ATL' To='PTY' Carrier='CM' Code='' />
			<NonStop From='ATL' To='PTY' Carrier='DL' Code='' />
			<NonStop From='ATL' To='PTY' Carrier='KE' Code='C' />
			<NonStop From='ATL' To='PTY' Carrier='KL' Code='C' />
			<NonStop From='AUA' To='PTY' Carrier='CM' Code='' />
			<NonStop From='AUS' To='PTY' Carrier='CM' Code='' />
			<NonStop From='AXM' To='PTY' Carrier='CM' Code='' />
			<NonStop From='BAQ' To='PTY' Carrier='CM' Code='' />
			<No

In [3]:
#using xml to read the data recived from the api
import xml.etree.ElementTree as et

In [4]:
data = et.fromstring(response.text)

In [8]:
#functions to extrasct desire information from the xml
def extract(data, param):
    empty = []
    for a in data.iter(param):
        empty.append(a.attrib)
    return empty


In [9]:
def extract_t(data, param):
    empty = []
    for a in data.iter(param):
        empty.append(a.text)
    return empty

In [10]:
#using functions
nonstops = extract(data, 'NonStop')
stop = extract(data, 'Route')
route = extract_t(data, 'Route')

In [32]:
#creating a data frame with the extracted data
df_stop = pd.DataFrame(stop)

In [46]:
#renaming columns
df_stop.rename(columns = {'From' : 'Departure', 'To' : 'Arrival'}, inplace=True)

In [38]:
df_stop['Route'] = route
df_stop

,From,To,Carrier,Hops,Route
0,GUA,PTY,CM,2,GUA-SJO-PTY
1,GUA,PTY,IB,2,GUA-SJO-PTY
2,IST,PTY,CM,2,IST-BOG-PTY
3,MGA,PTY,CM,2,MGA-GUA-PTY
4,MGA,PTY,CM,2,MGA-GUA-PTY
5,MGA,PTY,CM,2,MGA-SJO-PTY
6,PTY,GYE,AV,2,PTY-BOG-GYE
7,PTY,MGA,CM,2,PTY-GUA-MGA
8,PTY,MGA,CM,2,PTY-GUA-MGA
9,PTY,MGA,CM,2,PTY-SJO-MGA


In [47]:
#creating a second data frame
df_nonstops = pd.DataFrame(nonstops)
df_nonstops.rename(columns = {'From' : 'Departure', 'To' : 'Arrival'}, inplace=True)

In [57]:
#cleaning and formating data
df_nonstops['Route'] = df_nonstops['Departure'].astype(str) + "-" + df_nonstops['Arrival']

In [59]:
#rearenging columns order
columns = ['Route', 'Departure', 'Arrival', 'Carrier', 'Code']
df_nonstops = df_nonstops[columns]
columns_2 = ['Route', 'Departure', 'Arrival', 'Carrier']
df_stop = df_stop[columns_2]

In [64]:
#creating a csv 
df_stop.to_csv('./projects/Not_direct_routes.csv', index = False)
df_nonstops.to_csv('./projects/Direct_routes.csv', index = False)